In [1]:
from ultralytics import YOLOWorld, YOLO
import logging
import serial
import torch
import cv2
import time

# Load an official or custom model
# model = YOLO("yolo11x.pt")  # Load an official Detect model
# model = YOLOWorld('yolov8x-worldv2.pt')
# model.set_classes(['battery'])

model = YOLO("yoloe-11s-seg.mlpackage")

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = "mps"      # Apple Silicon GPU
elif torch.cuda.is_available():
    device = "cuda:0"   # NVIDIA GPU
else:
    device = "cpu"

# model = YOLO("yolo11n-seg.pt")  # Load an official Segment model
# model = YOLO("yolo11n-pose.pt")  # Load an official Pose model
# model = YOLO("path/to/best.pt")  # Load a custom trained model

# Perform tracking with the model
# results = model.track("https://youtu.be/LNwODJXcvt4", show=True)  # Tracking with default tracker

In [2]:
port = '/dev/tty.usbmodem2101'
serial_port = serial.Serial(port, 921600, timeout=0.1)

In [3]:
cv2.waitKey(1)

-1

# Data Collection

## Capture Image

In [ ]:
import cv2

# ========== CONFIGURATION ==========
# Set the output file name heres
filename = f"image.jpg"

# Set up the camera
cap = cv2.VideoCapture('http://192.168.1.27/stream')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)

if not cap.isOpened():
    raise

print("Reference Photo Capture Tool")
print("Position your object in the frame and press SPACE to capture")
print("Press ESC to exit")
print()

while True:
    # Capture frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Resize to ensure 800x800 if the camera doesn't support it directly
    frame = cv2.resize(frame, (800, 800))

    # Add instructions on screen
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "SPACE: Take Photo | ESC: Exit", (10, 30),
                font, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Capture Reference Photo", frame)

    key = cv2.waitKey(1) & 0xFF

    # Capture photo on spacebar
    if key == ord(' '):
        # Save photo
        cv2.imwrite(filename, frame)
        print(f"✅ Photo saved: {filename}")

        # Show confirmation
        confirm_frame = frame.copy()
        cv2.putText(confirm_frame, f"Saved: {filename}", (10, 70),
                    font, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("Capture Reference Photo", confirm_frame)
        cv2.waitKey(2000)

    # Exit on ESC
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Reference Photo Capture Tool
Position your object in the frame and press SPACE to capture
Press ESC to exit

✅ Photo saved: image.jpg


## Draw Box

In [2]:
import cv2

# ========== CONFIGURATION ==========
# Set your image path here:
image_path = "image.jpg"
# ===================================

class BoundingBoxTool:
    def __init__(self, image_path):
        self.image_path = image_path
        self.image = cv2.imread(image_path)
        if self.image is None:
            raise ValueError(f"❌ Could not load image: {image_path}")

        self.original_image = self.image.copy()
        self.boxes = []
        self.current_box = []
        self.drawing = False
        self.class_id = 0
        self.mouse_x = 0
        self.mouse_y = 0

        cv2.namedWindow("Bounding Box Tool", cv2.WINDOW_AUTOSIZE)
        cv2.setMouseCallback("Bounding Box Tool", self.mouse_callback)

    def mouse_callback(self, event, x, y, flags, param):
        self.mouse_x = x
        self.mouse_y = y

        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.current_box = [x, y]

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                temp_image = self.image.copy()
                cv2.rectangle(temp_image, (self.current_box[0], self.current_box[1]),
                            (x, y), (0, 255, 0), 2)
                self.draw_crosshairs(temp_image, x, y)
                cv2.imshow("Bounding Box Tool", temp_image)
            else:
                temp_image = self.image.copy()
                self.draw_crosshairs(temp_image, x, y)
                cv2.imshow("Bounding Box Tool", temp_image)

        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            self.current_box.extend([x, y])

            # Create bounding box [x1, y1, x2, y2]
            box = [
                min(self.current_box[0], self.current_box[2]),
                min(self.current_box[1], self.current_box[3]),
                max(self.current_box[0], self.current_box[2]),
                max(self.current_box[1], self.current_box[3])
            ]

            self.boxes.append({
                'bbox': box,
                'class_id': self.class_id
            })

            print(f"📦 Box {len(self.boxes)}: {box} (Class ID: {self.class_id})")
            self.class_id += 1
            self.update_display()

    def draw_crosshairs(self, image, x, y):
        height, width = image.shape[:2]
        cv2.line(image, (x, 0), (x, height), (255, 255, 0), 1)
        cv2.line(image, (0, y), (width, y), (255, 255, 0), 1)
        cv2.putText(image, f"({x}, {y})", (x + 10, y - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

    def update_display(self):
        self.image = self.original_image.copy()

        # Draw all boxes
        colors = [(0, 255, 0), (255, 0, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]
        for i, box_info in enumerate(self.boxes):
            bbox = box_info['bbox']
            class_id = box_info['class_id']
            color = colors[class_id % len(colors)]

            cv2.rectangle(self.image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
            cv2.putText(self.image, f"ID:{class_id}", (bbox[0], bbox[1]-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Add instructions
        cv2.putText(self.image, "R: Reset | Q: Quit", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(self.image, f"Next Class ID: {self.class_id}", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        if hasattr(self, 'mouse_x') and hasattr(self, 'mouse_y'):
            self.draw_crosshairs(self.image, self.mouse_x, self.mouse_y)

        cv2.imshow("Bounding Box Tool", self.image)

    def run(self):
        print(f"🎯 Bounding Box Tool - Loaded: {self.image_path}")
        print("📋 Instructions:")
        print("   • Click and drag to draw boxes around objects")
        print("   • Each box gets a unique Class ID (0, 1, 2...)")
        print("   • Press 'R' to reset all boxes")
        print("   • Press 'Q' to quit")
        print()

        self.update_display()

        while True:
            key = cv2.waitKey(1) & 0xFF

            if key == ord('q'):
                break
            elif key == ord('r'):
                self.boxes = []
                self.class_id = 0
                self.update_display()
                print("🔄 Reset all boxes")

        cv2.destroyAllWindows()

# Check if image exists
import os
if not os.path.exists(image_path):
    print(f"❌ Image not found: {image_path}")
    print("📁 Available images in current directory:")
    for file in os.listdir('.'):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            print(f"   {file}")
    exit()

# Run the tool
tool = BoundingBoxTool(image_path)
tool.run()

🎯 Bounding Box Tool - Loaded: image.jpg
📋 Instructions:
   • Click and drag to draw boxes around objects
   • Each box gets a unique Class ID (0, 1, 2...)
   • Press 'R' to reset all boxes
   • Press 'Q' to quit

📦 Box 1: [182, 302, 566, 729] (Class ID: 0)
🔄 Reset all boxes
📦 Box 1: [193, 347, 572, 719] (Class ID: 0)
🔄 Reset all boxes
🔄 Reset all boxes
📦 Box 1: [188, 314, 499, 595] (Class ID: 0)
🔄 Reset all boxes
📦 Box 1: [192, 304, 570, 718] (Class ID: 0)


## Training

In [6]:
from ultralytics import YOLOE
from ultralytics.models.yolo.yoloe import YOLOEVPSegPredictor
import numpy as np
import cv2

# ========== CONFIGURATION ==========
# Add as many objects as you want:
training_data = [
    {
        "image": "image.jpg",
        "box": [192, 304, 570, 718],
    },
    {
        "image": "battery.jpg",
        "box": [365, 315, 476, 520],
    }
]
# ===================================

model = YOLOE("yoloe-11s-seg.pt")

# Load all images and create a combined reference image
images = [cv2.imread(data["image"]) for data in training_data]

# Stack images horizontally to create combined reference
combined_image = np.hstack(images)

# Adjust bounding boxes for the combined image
all_bboxes = []
all_class_ids = []
x_offset = 0

for i, (data, img) in enumerate(zip(training_data, images)):
    box = data["box"].copy() if isinstance(data["box"], list) else list(data["box"])
    # Offset x coordinates by the cumulative width of previous images
    adjusted_box = [box[0] + x_offset, box[1], box[2] + x_offset, box[3]]
    all_bboxes.append(adjusted_box)
    all_class_ids.append(i)
    x_offset += img.shape[1]  # Add width of current image for next offset

visual_prompts = {
    'bboxes': np.array(all_bboxes),
    'cls': np.array(all_class_ids)
}

# Train with all objects at once using combined reference image
model.predict(
    training_data[0]["image"],  # Can use any image for initial prediction
    refer_image=combined_image,
    visual_prompts=visual_prompts,
    predictor=YOLOEVPSegPredictor,
    conf=0.1
)

model.export(format="coreml", imgsz=640)

print("Training complete!")
print("Object mapping:")
for i, data in enumerate(training_data):
    print(f"  ID {i}: {data['image']}")

Ultralytics 8.3.234 🚀 Python-3.11.14 torch-2.5.1 CPU (Apple M4 Pro)
YOLOe-11s-seg summary (fused): 137 layers, 13,693,398 parameters, 1,857,958 gradients

image 1/1 /Users/andrestorres/My Drive (actorres@ucsb.edu)/Fall 2025/ME225EH/YOLO/image.jpg: 640x640 1 object0, 79.7ms
Speed: 1.2ms preprocess, 79.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics 8.3.234 🚀 Python-3.11.14 torch-2.5.1 CPU (Apple M4 Pro)

PyTorch: starting from 'yoloe-11s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 38, 8400), (1, 32, 160, 160)) (26.5 MB)

CoreML: starting export with coremltools 9.0...


Tuple detected at graph output. This will be flattened in the converted model.
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 246.08 passes/s]


CoreML: export success ✅ 3.6s, saved as 'yoloe-11s-seg.mlpackage' (19.4 MB)

Export complete (3.7s)
Results saved to /Users/andrestorres/My Drive (actorres@ucsb.edu)/Fall 2025/ME225EH/YOLO
Predict:         yolo predict task=segment model=yoloe-11s-seg.mlpackage imgsz=640  
Validate:        yolo val task=segment model=yoloe-11s-seg.mlpackage imgsz=640 data={'train': {'yolo_data': ['Objects365v1.yaml'], 'grounding_data': [{'img_path': '../datasets/flickr/full_images/', 'json_file': '../datasets/flickr/annotations/final_flickr_separateGT_train_segm.json'}, {'img_path': '../datasets/mixed_grounding/gqa/images', 'json_file': '../datasets/mixed_grounding/annotations/final_mixed_train_no_coco_segm.json'}]}, 'val': {'yolo_data': ['lvis.yaml']}}  
Visualize:       https://netron.app
Training complete!
Object mapping:
  ID 0: image.jpg
  ID 1: battery.jpg


# Serial Testing

In [38]:
serial_port.write('P9:0\n'.encode())
time.sleep(0.1)
serial_port.write('P10:0\n'.encode())

6

In [ ]:
serial_port.write('P10:1\n'.encode())
time.sleep(0.02)
serial_port.write('P10:0\n'.encode())

6

In [ ]:
serial_port.write('P10:1\n'.encode())
time.sleep(0.01)
serial_port.write('P9:0\n'.encode())

5

In [37]:
serial_port.write('P10:0\n'.encode())
time.sleep(0.02)
serial_port.write('P9:0\n'.encode())

5

# Websocket Servo Control

In [61]:
#!/usr/bin/env python

"""Client using the threading API."""

from websockets.sync.client import connect
import json

with connect("ws://192.168.1.27/websocket") as websocket:
    websocket.send(json.dumps({"angle": 0}))
    # message = websocket.recv()
    # print(message)


# Predict
(use this!)

0.2 for the area of the 

In [ ]:
from typing import Literal
from concurrent.futures import ThreadPoolExecutor
from websockets import

def direction(serial_: serial.Serial, state: Literal['left', 'right', 'middle'] = 'middle'):
    # print(f'Moving: {state}    ', end='\r')
    if state == 'left':
        serial_.write('P10:0\n'.encode())
        time.sleep(0.2)
        serial_.write('P9:1\n'.encode())
    elif state == 'right':
        serial_.write('P10:1\n'.encode())
        time.sleep(0.2)
        serial_.write('P9:0\n'.encode())
    else:
        serial_.write('P10:0\n'.encode())
        time.sleep(0.2)
        serial_.write('P9:0\n'.encode())

state = 'middle'
end = False

def direction_thread():
    global state
    while not end:
        direction(serial_port, state)
        time.sleep(0.5)
        direction(serial_port)
        time.sleep(1)
try:
    logging.getLogger('ultralytics').setLevel(logging.ERROR)

    with ThreadPoolExecutor() as executor, connect("ws://192.168.1.27/websocket") as websocket:
        executor.submit(direction_thread)
        for result in model.predict(
            'http://192.168.1.27/stream', stream=True,
            device='mps',
            verbose=False,
        ):
            frame = result.plot()

            # Get inference time
            text = f"FPS: {1000 / result.speed['inference']:.1f}"

            # Define font and position
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_size = cv2.getTextSize(text, font, 1, 2)[0]
            text_x = frame.shape[1] - text_size[0] - 10  # 10 pixels from the right
            text_y = text_size[1] + 10  # 10 pixels from the top

            cv2.putText(frame, text, (text_x, text_y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

            if result.boxes:
                battery = result.boxes[0]
                if battery:
                    x, y, w, h = battery.xywhn.tolist()[0]

                    area = w * h
                    if area > 0.2:
                        websocket.send(json.dumps({"angle": 0}))

                    if x < 0.4:
                        state = 'left'
                    elif x > 0.6:
                        state = 'right'
                    else:
                        state = 'middle'

                    font = cv2.FONT_HERSHEY_SIMPLEX
                    text_size = cv2.getTextSize(state, font, 1, 2)[0]
                    text_x = 10  # 10 pixels from the left
                    text_y = text_size[1] + 10  # 10 pixels from the top

                    cv2.putText(frame, f'{state} area: {w*h:.3f}', (text_x, text_y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.imshow("Battery Tracking", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                end = True
                break
finally:
    end = True
    cv2.destroyAllWindows()
    cv2.waitKey(1)

[ WARN:3@2624.470] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30031.166046 ms
[ WARN:3@2624.470] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30031.314296 ms
[ WARN:3@2624.470] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30031.364046 ms
[ WARN:3@2624.470] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30031.397921 ms
[ WARN:4@2624.493] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30015.255339 ms
[ WARN:4@2624.493] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30015.374214 ms
[ WARN:4@2624.493] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30015.423131 ms
[ WARN:4@2624.493] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback

In [33]:
cv2.waitKey(1)

-1

# Tracking
(don't use this it doesn't work super well)

In [ ]:
import itertools

try:
    tracking_id = None
    counter = itertools.count()
    for result in model.track(
        'http://192.168.1.27/stream', stream=True,
        device='cpu',
        verbose=False,
        persist=True
    ):
        frame = result.plot()
        if result.boxes and result.boxes.is_track:
            if tracking_id is None:
                tracking_id = result.boxes.id.int().cpu().tolist()[0]
                print(f"Initialized tracking ID: {tracking_id}")
            battery = result.boxes[result.boxes.id == tracking_id]
            if battery:
                x = battery.xywhn[0, 0]
                if x < 0.4:
                    print("Battery is on the left side.")
                elif x > 0.6:
                    print("Battery is on the right side.")
            else:
                if next(counter) % 30 == 0:
                    print("Battery lost resetting the tracking id.")
                    tracking_id = None  # Reset tracking ID if battery not found for a while
        cv2.imshow("Battery Tracking", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
                break

finally:
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
# Create a VideoCapture object
# cap = cv2.VideoCapture('http://192.168.1.28/stream')
cap = cv2.VideoCapture(0)

# Check if the stream opened successfully
if not cap.isOpened():
    print("Error: Could not open MJPEG stream.")
else:
    while True:
        # Read a frame from the stream
        ret, frame = cap.read()

        # If frame is read correctly, ret is True
        if not ret:
            print("Error: Could not read frame from stream. Exiting.")
            break

        # Display the frame
        cv2.imshow('MJPEG Stream', frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object and destroy all windows
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
cap = cv2.VideoCapture(0)
success, frame = cap.read()
result = model.track(frame, tracker="bytetrack.yaml", device=device, persist=True)
result.save('test.jpg')
result.show()
cap.release()


0: 384x640 2 batterys, 46.3ms
Speed: 3.3ms preprocess, 46.3ms inference, 35.5ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
from collections import defaultdict

import cv2
import numpy as np

# Open the video file
# video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(0)

# Store the track history
track_history = defaultdict(lambda: [])
try:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        result = model.track(frame, tracker="bytetrack.yaml", device=device, persist=True)[0]

        # Get the boxes and track IDs
        if result.boxes and result.boxes.is_track:
            boxes = result.boxes.xywh.cpu()
            track_ids = result.boxes.id.int().cpu().tolist()

            # Visualize the result on the frame
            frame = result.plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [12]:
cv2.waitKey(1)

-1

In [19]:
type(results[-2].boxes)

ultralytics.engine.results.Boxes

In [28]:
results[-2].boxes[0]  # x1, y1, x2, y2

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='mps:0')
conf: tensor([0.2499], device='mps:0')
data: tensor([[6.5317e+02, 4.5461e+02, 8.4469e+02, 7.7824e+02, 2.4985e-01, 0.0000e+00]], device='mps:0')
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[748.9326, 616.4263, 191.5153, 323.6240]], device='mps:0')
xywhn: tensor([[0.3901, 0.5708, 0.0997, 0.2997]], device='mps:0')
xyxy: tensor([[653.1749, 454.6143, 844.6902, 778.2382]], device='mps:0')
xyxyn: tensor([[0.3402, 0.4209, 0.4399, 0.7206]], device='mps:0')